In [0]:
import pandas as pd
from pyspark.sql import functions as F, Window
from pyspark.sql.functions import round, asc, when, to_date, date_format, regexp_replace, col, sum

In [0]:
df1 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/pradeepti239@gmail.com/fellowship_data.csv")

In [0]:
df1.show(50)

+--------+------------+-----+--------+--------+
|Duration|        Date|Pulse|Maxpulse|Calories|
+--------+------------+-----+--------+--------+
|      60|'2020/12/01'|  110|     130|   409.1|
|      60|'2020/12/02'|  117|     145|   479.0|
|      60|'2020/12/03'|  103|     135|   340.0|
|      45|'2020/12/04'|  109|     175|   282.4|
|      45|'2020/12/05'|  117|     148|   406.0|
|      60|'2020/12/06'|  102|     127|   300.0|
|      60|'2020/12/07'|  110|     136|   374.0|
|     450|'2020/12/08'|  104|     134|   253.3|
|      30|'2020/12/09'|  109|     133|   195.1|
|      60|'2020/12/10'|   98|     124|   269.0|
|      60|'2020/12/11'|  103|     147|   329.3|
|      60|'2020/12/12'|  100|     120|   250.7|
|      60|'2020/12/12'|  100|     120|   250.7|
|      60|'2020/12/13'|  106|     128|   345.3|
|      60|'2020/12/14'|  104|     132|   379.3|
|      60|'2020/12/15'|   98|     123|   275.0|
|      60|'2020/12/16'|   98|     120|   215.2|
|      60|'2020/12/17'|  100|     120|  

In [0]:
# Checking duplicates

df1 \
    .groupby(['Duration', 'Date','Pulse','Maxpulse','Calories']) \
    .count() \
    .where('count > 1') \
    .sort('count', ascending=False) \
    .show()

+--------+------------+-----+--------+--------+-----+
|Duration|        Date|Pulse|Maxpulse|Calories|count|
+--------+------------+-----+--------+--------+-----+
|      60|'2020/12/12'|  100|     120|   250.7|    2|
+--------+------------+-----+--------+--------+-----+



In [0]:
# Dropping duplicates

df1 = df1.dropDuplicates()
df1.show(50)

+--------+------------+-----+--------+--------+
|Duration|        Date|Pulse|Maxpulse|Calories|
+--------+------------+-----+--------+--------+
|      45|        null|  100|     119|   282.0|
|      60|'2020/12/25'|  102|     126|   334.5|
|      60|'2020/12/15'|   98|     123|   275.0|
|      60|'2020/12/21'|  108|     131|   364.2|
|      60|    20201226|  100|     120|   250.0|
|      60|'2020/12/01'|  110|     130|   409.1|
|      45|'2020/12/24'|  105|     132|   246.0|
|      60|'2020/12/16'|   98|     120|   215.2|
|      60|'2020/12/17'|  100|     120|   300.0|
|      60|'2020/12/31'|   92|     115|   243.0|
|      60|'2020/12/11'|  103|     147|   329.3|
|      60|'2020/12/10'|   98|     124|   269.0|
|      60|'2020/12/02'|  117|     145|   479.0|
|     450|'2020/12/08'|  104|     134|   253.3|
|      60|'2020/12/12'|  100|     120|   250.7|
|      60|'2020/12/19'|  103|     123|   323.0|
|      45|'2020/12/04'|  109|     175|   282.4|
|      45|'2020/12/20'|   97|     125|  

In [0]:
# Checking if duplicate is dropped or not

df1 \
    .groupby(['Duration', 'Date','Pulse','Maxpulse','Calories']) \
    .count() \
    .where('count > 1') \
    .sort('count', ascending=False) \
    .show()

+--------+----+-----+--------+--------+-----+
|Duration|Date|Pulse|Maxpulse|Calories|count|
+--------+----+-----+--------+--------+-----+
+--------+----+-----+--------+--------+-----+



In [0]:
# Replacing anomalous values in Duration column (450) by the mode of that column

upper = 60
lower = 45

df1 = df1.withColumn(
    'Duration', 
    F.when(
        (df1['Duration'] > upper) | (df1['Duration'] < lower), 
        F.round(F.mode('Duration').over(Window.orderBy(F.lit(1)))).cast('int')
    ).otherwise(F.col('Duration'))
)

df1.show(50)


+--------+------------+-----+--------+--------+
|Duration|        Date|Pulse|Maxpulse|Calories|
+--------+------------+-----+--------+--------+
|      45|        null|  100|     119|   282.0|
|      60|'2020/12/25'|  102|     126|   334.5|
|      60|'2020/12/15'|   98|     123|   275.0|
|      60|'2020/12/21'|  108|     131|   364.2|
|      60|    20201226|  100|     120|   250.0|
|      60|'2020/12/01'|  110|     130|   409.1|
|      45|'2020/12/24'|  105|     132|   246.0|
|      60|'2020/12/16'|   98|     120|   215.2|
|      60|'2020/12/17'|  100|     120|   300.0|
|      60|'2020/12/31'|   92|     115|   243.0|
|      60|'2020/12/11'|  103|     147|   329.3|
|      60|'2020/12/10'|   98|     124|   269.0|
|      60|'2020/12/02'|  117|     145|   479.0|
|      60|'2020/12/08'|  104|     134|   253.3|
|      60|'2020/12/12'|  100|     120|   250.7|
|      60|'2020/12/19'|  103|     123|   323.0|
|      45|'2020/12/04'|  109|     175|   282.4|
|      45|'2020/12/20'|   97|     125|  

In [0]:
# Pulse Difference column shows the difference between Maxpulse and Pulse

df1= df1.withColumn("Pulse Difference",df1['Maxpulse']-df1['Pulse'])
df1.show(50)

+--------+------------+-----+--------+--------+----------------+
|Duration|        Date|Pulse|Maxpulse|Calories|Pulse Difference|
+--------+------------+-----+--------+--------+----------------+
|      45|        null|  100|     119|   282.0|            19.0|
|      60|'2020/12/25'|  102|     126|   334.5|            24.0|
|      60|'2020/12/15'|   98|     123|   275.0|            25.0|
|      60|'2020/12/21'|  108|     131|   364.2|            23.0|
|      60|    20201226|  100|     120|   250.0|            20.0|
|      60|'2020/12/01'|  110|     130|   409.1|            20.0|
|      45|'2020/12/24'|  105|     132|   246.0|            27.0|
|      60|'2020/12/16'|   98|     120|   215.2|            22.0|
|      60|'2020/12/17'|  100|     120|   300.0|            20.0|
|      60|'2020/12/31'|   92|     115|   243.0|            23.0|
|      60|'2020/12/11'|  103|     147|   329.3|            44.0|
|      60|'2020/12/10'|   98|     124|   269.0|            26.0|
|      60|'2020/12/02'|  

In [0]:
# Calories Burned column shows the calories burned per second in 2 decimal places

df1= df1.withColumn("Calories Burned",round(df1['Calories']/df1['Duration'],2))
df1.show(50)

+--------+------------+-----+--------+--------+----------------+---------------+
|Duration|        Date|Pulse|Maxpulse|Calories|Pulse Difference|Calories Burned|
+--------+------------+-----+--------+--------+----------------+---------------+
|      45|        null|  100|     119|   282.0|            19.0|           6.27|
|      60|'2020/12/25'|  102|     126|   334.5|            24.0|           5.58|
|      60|'2020/12/15'|   98|     123|   275.0|            25.0|           4.58|
|      60|'2020/12/21'|  108|     131|   364.2|            23.0|           6.07|
|      60|    20201226|  100|     120|   250.0|            20.0|           4.17|
|      60|'2020/12/01'|  110|     130|   409.1|            20.0|           6.82|
|      45|'2020/12/24'|  105|     132|   246.0|            27.0|           5.47|
|      60|'2020/12/16'|   98|     120|   215.2|            22.0|           3.59|
|      60|'2020/12/17'|  100|     120|   300.0|            20.0|            5.0|
|      60|'2020/12/31'|   92

In [0]:
# Sorting df by Calories Column

df1_sorted = df1.orderBy(asc('Calories'))

df1_sorted.show(50)

+--------+------------+-----+--------+--------+----------------+---------------+
|Duration|        Date|Pulse|Maxpulse|Calories|Pulse Difference|Calories Burned|
+--------+------------+-----+--------+--------+----------------+---------------+
|      45|'2020/12/18'|   90|     112|    null|            22.0|           null|
|      60|'2020/12/28'|  103|     132|    null|            29.0|           null|
|      60|'2020/12/09'|  109|     133|   195.1|            24.0|           3.25|
|      60|'2020/12/16'|   98|     120|   215.2|            22.0|           3.59|
|      60|'2020/12/27'|   92|     118|   241.0|            26.0|           4.02|
|      60|'2020/12/31'|   92|     115|   243.0|            23.0|           4.05|
|      45|'2020/12/20'|   97|     125|   243.0|            28.0|            5.4|
|      45|'2020/12/24'|  105|     132|   246.0|            27.0|           5.47|
|      60|    20201226|  100|     120|   250.0|            20.0|           4.17|
|      60|'2020/12/12'|  100

In [0]:
#  replacing the substring with single quotation ('') i.e., '(.*?)' with the subsrting itself i.e., $1

df1 = df1.withColumn("Date", regexp_replace("Date", "'(.*?)'", "$1"))

In [0]:
# converting value 20201226 into 2020/12/26
df = df1.withColumn('Date', when(to_date(df1['Date'],'yyyyMMdd').isNotNull(), date_format(to_date(df1['Date'], 'yyyyMMdd'),'yyyy/MM/dd')).otherwise(df1['Date']))

df.show(50)

+--------+----------+-----+--------+--------+----------------+---------------+
|Duration|      Date|Pulse|Maxpulse|Calories|Pulse Difference|Calories Burned|
+--------+----------+-----+--------+--------+----------------+---------------+
|      45|      null|  100|     119|   282.0|            19.0|           6.27|
|      60|2020/12/25|  102|     126|   334.5|            24.0|           5.58|
|      60|2020/12/15|   98|     123|   275.0|            25.0|           4.58|
|      60|2020/12/21|  108|     131|   364.2|            23.0|           6.07|
|      60|2020/12/26|  100|     120|   250.0|            20.0|           4.17|
|      60|2020/12/01|  110|     130|   409.1|            20.0|           6.82|
|      45|2020/12/24|  105|     132|   246.0|            27.0|           5.47|
|      60|2020/12/16|   98|     120|   215.2|            22.0|           3.59|
|      60|2020/12/17|  100|     120|   300.0|            20.0|            5.0|
|      60|2020/12/31|   92|     115|   243.0|       

In [0]:
# Retrieve a subset of the dataframes where calories are higher than 400

df2 = df.where(df['Calories'] > 400)

df2.show(50)

+--------+----------+-----+--------+--------+----------------+---------------+
|Duration|      Date|Pulse|Maxpulse|Calories|Pulse Difference|Calories Burned|
+--------+----------+-----+--------+--------+----------------+---------------+
|      60|2020/12/01|  110|     130|   409.1|            20.0|           6.82|
|      60|2020/12/02|  117|     145|   479.0|            28.0|           7.98|
|      45|2020/12/05|  117|     148|   406.0|            31.0|           9.02|
+--------+----------+-----+--------+--------+----------------+---------------+



In [0]:
# Retrieve a subset of the dataframes where calories are less than 200

df2 = df.where(df['Calories'] < 200)

df2.show(50)

+--------+----------+-----+--------+--------+----------------+---------------+
|Duration|      Date|Pulse|Maxpulse|Calories|Pulse Difference|Calories Burned|
+--------+----------+-----+--------+--------+----------------+---------------+
|      60|2020/12/09|  109|     133|   195.1|            24.0|           3.25|
+--------+----------+-----+--------+--------+----------------+---------------+



In [0]:
# Retrieve a subset of the dataframes where calories are between 200-300.
df2 = df.where((df['Calories'] > 200) & (df['Calories'] < 300))

df2.show(50)

+--------+----------+-----+--------+--------+----------------+---------------+
|Duration|      Date|Pulse|Maxpulse|Calories|Pulse Difference|Calories Burned|
+--------+----------+-----+--------+--------+----------------+---------------+
|      45|      null|  100|     119|   282.0|            19.0|           6.27|
|      60|2020/12/15|   98|     123|   275.0|            25.0|           4.58|
|      60|2020/12/26|  100|     120|   250.0|            20.0|           4.17|
|      45|2020/12/24|  105|     132|   246.0|            27.0|           5.47|
|      60|2020/12/16|   98|     120|   215.2|            22.0|           3.59|
|      60|2020/12/31|   92|     115|   243.0|            23.0|           4.05|
|      60|2020/12/10|   98|     124|   269.0|            26.0|           4.48|
|      60|2020/12/08|  104|     134|   253.3|            30.0|           4.22|
|      60|2020/12/12|  100|     120|   250.7|            20.0|           4.18|
|      45|2020/12/04|  109|     175|   282.4|       

In [0]:
# Find Unique values in duration

unique_values = df.select('duration').distinct()
unique_values.show()

+--------+
|duration|
+--------+
|      45|
|      60|
+--------+



In [0]:
# Count how many missing values are there in the dataframe
missing_count = df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns))

missing_count.show()

+--------+----+-----+--------+--------+----------------+---------------+
|Duration|Date|Pulse|Maxpulse|Calories|Pulse Difference|Calories Burned|
+--------+----+-----+--------+--------+----------------+---------------+
|       0|   1|    0|       0|       2|               0|              2|
+--------+----+-----+--------+--------+----------------+---------------+

